In [ ]:
# import necessary libraries

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import glob
from skimage.filters.rank import entropy
from skimage.morphology import disk
from skimage import io
from skimage.filters import threshold_otsu

In [ ]:
def starting_time(path):
    """
    Measures the gap areas in all the images in a folder
    
    Parameters
    ----------
    path: path of the folder containing the images from time 0.
    
    Returns
    ----------
    A list containing the gap areas of the images in order.
    """

    time = 0
    time_0 = []

    for file in glob.glob(path):
    
        img = io.imread(file, as_gray = True)
        entropy_img = entropy(img, disk(10))
        thresh = threshold_otsu(entropy_img)
        binary = entropy_img <= thresh
        time_0.append(np.sum(binary == 1))
        time += 1
        
    return time_0